# Use an LLM agent to query the macula atlas api 

Using curl, this can be accomplished with
```
curl 'https://macula-atlas-api-qa-25c5xl4maa-uk.a.run.app/graphql/' \
-H 'Content-Type: application/json' \
-H 'Accept: application/json' \
--compressed \
--data-binary '{
  "query": "query IntrospectionQuery { __schema { types { name kind fields { name type { name kind ofType { name kind } } } } } }"
}'
```

The following code does this in a more pythonic way, and reformats the output as yaml for easier LLM processing.

In [8]:
import requests

import requests
# import yaml # NOTE: for testing whether YAML schema is more efficient token-wise

def get_macula_atlas_schema():
    """Query the macula atlas api for its schema""" 
    query = """
    query IntrospectionQuery {
        __schema {
            types {
                name
                kind
                fields {
                    name
                    type {
                        name
                        kind
                        ofType {
                            name
                            kind
                        }
                    }
                }
            }
        }
    }"""
    endpoint = "https://macula-atlas-api-qa-25c5xl4maa-uk.a.run.app/graphql/"
    request = requests.post(endpoint, json={'query': query})
    json_output = request.json()

    # Simplify the schema
    simplified_schema = {}
    for type_info in json_output['data']['__schema']['types']:
        
         if not type_info['name'].startswith('__'):
            
            fields = type_info.get('fields')
            if fields is not None and fields is not []:
                simplified_schema[type_info['name']] = {
                    'kind': type_info['kind'],
                    'fields': ', '.join([field['name'] for field in fields if not field['name'].startswith('__')])
                }
            else:
                simplified_schema[type_info['name']] = {
                    'kind': type_info['kind'],
                }
                
    return simplified_schema

    # Convert the simplified schema to YAML
    # yaml_output = yaml.dump(simplified_schema, default_flow_style=False)

    # return yaml_output



In [9]:
print(get_macula_atlas_schema())

{'SiteQuery': {'kind': 'OBJECT', 'fields': 'passage, wordTokens, annotationSets, annotationFeatures, annotations'}, 'Passage': {'kind': 'OBJECT', 'fields': 'id, textContent, depth, ref, usfmRef, tokens, textualEdition, xmlContent'}, 'ID': {'kind': 'SCALAR'}, 'String': {'kind': 'SCALAR'}, 'Int': {'kind': 'SCALAR'}, 'WordToken': {'kind': 'OBJECT', 'fields': 'id, value, verse, annotationInstances, data, lemma, maculaId, ref, textualEdition, wordValue, xmlId'}, 'OffsetPaginationInput': {'kind': 'INPUT_OBJECT'}, 'Annotation': {'kind': 'OBJECT', 'fields': 'id, label, depth, data, uri, annotationSet, feature, tokens'}, 'JSON': {'kind': 'SCALAR'}, 'AnnotationSet': {'kind': 'OBJECT', 'fields': 'id, label, data, uri, features'}, 'AnnotationFeature': {'kind': 'OBJECT', 'fields': 'id, label, data, uri, annotationSet, instances'}, 'AnnotationFilter': {'kind': 'INPUT_OBJECT'}, 'StrFilterLookup': {'kind': 'INPUT_OBJECT'}, 'Boolean': {'kind': 'SCALAR'}, 'AnnotationFeatureFilter': {'kind': 'INPUT_OBJEC

In [10]:
import os
import getpass
secret_key = getpass.getpass('Enter OpenAI secret key: ')
os.environ['OPENAI_API_KEY'] = secret_key

In [5]:
!pip3.10 install httpx gql

  Using cached gql-3.4.1-py2.py3-none-any.whl (65 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl (202 kB)


In [13]:
from langchain import OpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.utilities import GraphQLAPIWrapper

llm = OpenAI(temperature=0)

tools = load_tools(
    ["graphql"],
    graphql_endpoint="https://macula-atlas-api-qa-25c5xl4maa-uk.a.run.app/graphql/",
    llm=llm,
)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [17]:
graphql_fields = get_macula_atlas_schema()

query = "What are the discourse features of MAT 5:3?"

examples = """
query = query AnnotationFeatures($filters1: AnnotationFeatureFilter, $filters2: AnnotationFilter) {
  annotationFeatures(filters: $filters1) {
   label
    uri
    instances(filters: $filters2) {
      uri
      tokens {
        ref
      }
    }
  }
}

variables = {
  "filters1": {
    "reference": "2CO 8:2"
  },
  "filters2": {
    "reference": "2CO 8:2"
  }
}
"""

prompt = f"""Here is an example query for the graphql endpoint described below:
{examples}

Answer the following question: {query} in the graphql database that has this schema {graphql_fields}"""

agent.run(prompt)

Error in on_chain_start callback: 'name'


 I need to query the graphql endpoint to get the discourse features of MAT 5:3
Action: query_graphql
Action Input: query {
  annotationFeatures(filters: {reference: "MAT 5:3"}) {
    label
    uri
    instances(filters: {reference: "MAT 5:3"}) {
      uri
      tokens {
        ref
      }
    }
  }
}
Observation: "{\n  \"annotationFeatures\": [\n    {\n      \"label\": \"Main clauses\",\n      \"uri\": \"https://github.com/biblicalhumanities/levinsohn:main-clauses\",\n      \"instances\": [\n        {\n          \"uri\": \"https://github.com/biblicalhumanities/levinsohn:main-clauses.320\",\n          \"tokens\": [\n            {\n              \"ref\": \"MAT 5:3!1\"\n            },\n            {\n              \"ref\": \"MAT 5:3!2\"\n            },\n            {\n              \"ref\": \"MAT 5:3!3\"\n            },\n            {\n              \"ref\": \"MAT 5:3!4\"\n            },\n            {\n              \"ref\": \"MAT 5:3!5\"\n            }\n          ]\n        },\n       

'The discourse features of MAT 5:3 are Main clauses and annotations.'